In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
import tensorflow_datasets as tfds

import numpy as np
import pandas as pd
import os
import PIL
import PIL.Image
import glob

import cv2

import matplotlib.pyplot as plt
from matplotlib import gridspec

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", len(physical_devices))

Num GPUs: 0


2022-09-22 16:33:11.476924: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2022-09-22 16:33:11.476944: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: mike-System-Product-Name
2022-09-22 16:33:11.476948: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: mike-System-Product-Name
2022-09-22 16:33:11.477033: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 515.65.1
2022-09-22 16:33:11.477043: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 515.65.1
2022-09-22 16:33:11.477045: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 515.65.1


In [250]:
ds_train = image_dataset_from_directory(
'tiny_imagenet/train',
image_size=(64,64),
batch_size=32)

Found 100000 files belonging to 200 classes.


In [244]:
ds_val = image_dataset_from_directory(
'tiny_imagenet/val',
image_size=(64,64),
batch_size=32)

Found 10000 files belonging to 200 classes.


In [189]:
# conv_base = keras.applications.vgg19.VGG19(
# weights="imagenet",
# input_shape=(64, 64, 3),
# include_top=False)

# conv_base.trainable = False
# for layer in conv_base.layers:
#     if layer.name == 'block5_conv1':
#         set_trainable = True
#     if set_trainable:
#         layer.trainable = True
#     else:
#         layer.trainable = False

In [252]:
inputs = keras.Input(shape=(64,64,3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=5, use_bias=False)(x)

for size in [32,64,128,256,512]:
    residual = x
    
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

    residual = layers.Conv2D(size, 1, strides=2, padding="same", use_bias=False)(residual)
    x = layers.add([x, residual])


x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(200, activation="softmax")(x)
model = keras.Model(inputs, outputs)

In [253]:
opt = tf.optimizers.Adam(learning_rate=0.0001)

In [254]:
model.compile(
    optimizer=opt,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [255]:
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="tiny_imagenet_feature_extraction_without_data_augmentation.keras",
save_best_only=True,
monitor="val_loss")
]

In [256]:
history = model.fit(
ds_train,
epochs=40,
batch_size = 32,
validation_data=ds_val,
callbacks=callbacks)

Epoch 1/100
3125/3125 [==============================] - 106s 34ms/step - loss: 4.9937 - accuracy: 0.0266 - val_loss: 4.5651 - val_accuracy: 0.0614
Epoch 2/100
3125/3125 [==============================] - 107s 34ms/step - loss: 4.4931 - accuracy: 0.0657 - val_loss: 4.1741 - val_accuracy: 0.1081
Epoch 3/100
3125/3125 [==============================] - 106s 34ms/step - loss: 4.1956 - accuracy: 0.1007 - val_loss: 3.9071 - val_accuracy: 0.1447
Epoch 4/100
3125/3125 [==============================] - 104s 33ms/step - loss: 3.9812 - accuracy: 0.1263 - val_loss: 3.9163 - val_accuracy: 0.1456
Epoch 5/100
3125/3125 [==============================] - 76s 24ms/step - loss: 3.8156 - accuracy: 0.1513 - val_loss: 3.6979 - val_accuracy: 0.1751
Epoch 6/100
3125/3125 [==============================] - 77s 25ms/step - loss: 3.6783 - accuracy: 0.1727 - val_loss: 3.5247 - val_accuracy: 0.2030
Epoch 7/100
3125/3125 [==============================] - 90s 29ms/step - loss: 3.5603 - accuracy: 0.1933 - val_los

InternalError: Could not synchronize CUDA stream: CUDA_ERROR_LAUNCH_FAILED: unspecified launch failure

In [257]:
data_augmentation = keras.Sequential(
[
layers.RandomFlip("horizontal"),
layers.RandomRotation(0.1),
layers.RandomZoom(0.2),
]
)

: 

: 

In [ ]:
inputs = keras.Input(shape=(64,64,3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=5, use_bias=False)(x)

for size in [32,64,128,256,512]:
    residual = x
    
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

    residual = layers.Conv2D(size, 1, strides=2, padding="same", use_bias=False)(residual)
    x = layers.add([x, residual])


x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(200, activation="softmax")(x)
model = keras.Model(inputs, outputs)

In [ ]:
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="tiny_imagenet_feature_extraction_with_data_augmentation.keras",
save_best_only=True,
monitor="val_loss")
]

In [ ]:
history = model.fit(
ds_train,
epochs=40,
batch_size = 32,
validation_data=ds_val,
callbacks=callbacks)